In [ ]:
!pip install pyyaml
!pip install dndice
!pip install python-i18n
!pip install gymnasium
!pip install inflect
!pip install collections-extended
!pip install -e ..


In [ ]:
import unittest
from natural20.map import Map, Terrain
from natural20.battle import Battle
from natural20.player_character import PlayerCharacter
from natural20.map_renderer import MapRenderer
from natural20.die_roll import DieRoll
from natural20.generic_controller import GenericController
from natural20.utils.utils import Session
from natural20.actions.move_action import MoveAction
from natural20.action import Action
from natural20.gym.dndenv import dndenv
from gymnasium import register, envs, make
import tqdm as tqdm
import random

In [ ]:
env = make("dndenv-v0", root_path="../templates")
print(env.observation_space.shape)
print(env.action_space.shape)

In [ ]:
def generate_trajectory(env, policy="e-greedy", epsilon=0.1, num_episodes=1000):
    state, info = env.reset()
    done = False
    truncated = False
    trajectory = []
    while not done and not truncated:
        action = random.choice(info["available_moves"])
        observation, reward, done, truncated, info = env.step(action)
        trajectory.append((observation, reward, done, info))
    return trajectory

In [ ]:
EPIDODES = 100
avg_reward = 0
for i in tqdm.tqdm(range(EPIDODES)):
    trajectory = generate_trajectory(env)
    total_reward = 0
    for observation, reward, done, info in trajectory:
        total_reward += reward
    avg_reward += total_reward

avg_reward /= EPIDODES
print(f"Average reward: {avg_reward}")